In [1]:
from __future__  import print_function  # Python 2/3 compatibility

import pkg_resources
import json
import pandas as pd
import boto3
pkg_resources.get_distribution("boto3").version

'1.19.3'

In [2]:
s3 = boto3.client('s3')

response = s3.get_object(
    Bucket='bearly-graph',
    Key='src/mpd.slice.0-999.json',
)
# Extracts body of the 
body = response['Body']
str_body = body.read()
s3_obj_content = json.loads(str_body)
playlists = s3_obj_content['playlists']

In [3]:
edge_list = []

for playlist in playlists:
    tracks = playlist['tracks']
    for track in tracks:
        # Track to Playlist
        edge_dict = {}
        edge_dict['~id'] = track['track_uri'] +'-'+str(playlist['pid'])
        edge_dict['~from'] = track['track_uri']
        edge_dict['~to'] = 'playlist_'+str(playlist['pid'])
        edge_dict['~label'] = 'trackin'
        edge_list.append(edge_dict)
        # Track to Album
        edge_dict = {}
        edge_dict['~id'] = track['track_uri'] +'-'+ track['album_uri']
        edge_dict['~from'] = track['track_uri']
        edge_dict['~to'] = track['album_uri']
        edge_dict['~label'] = 'trackin'
        edge_list.append(edge_dict)
        # Artist to Track 
        edge_dict = {}
        edge_dict['~id'] = track['artist_uri'] +'-'+ track['track_uri'] 
        edge_dict['~from'] = track['artist_uri']
        edge_dict['~to'] = track['track_uri']
        edge_dict['~label'] = 'madeby'
        edge_list.append(edge_dict)
        # Artist to Album 
        edge_dict = {}
        edge_dict['~id'] = track['artist_uri'] +'-'+ track['album_uri'] 
        edge_dict['~from'] = track['artist_uri']
        edge_dict['~to'] = track['album_uri']
        edge_dict['~label'] = 'madeby'
        edge_list.append(edge_dict)
        # Artist to Track 
        edge_dict = {}
        edge_dict['~id'] = track['artist_uri'] +'-'+str(playlist['pid'])
        edge_dict['~from'] = track['artist_uri']
        edge_dict['~to'] = 'playlist_'+str(playlist['pid'])
        edge_dict['~label'] = 'artistin'
        edge_list.append(edge_dict)
        # Album to Playlist
        edge_dict = {}
        edge_dict['~id'] = track['album_uri'] +'-'+ str(playlist['pid'])
        edge_dict['~from'] = track['artist_uri']
        edge_dict['~to'] =  'playlist_'+str(playlist['pid'])
        edge_dict['~label'] = 'albumin'
        edge_list.append(edge_dict)

df = pd.DataFrame.from_dict(edge_list)



In [4]:
df = df.drop_duplicates()
df

,~id,~from,~to,~label
0,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI-0,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,playlist_0,trackin
1,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI-spotify:a...,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,trackin
2,spotify:artist:2wIVse2owClT7go1WT98tk-spotify:...,spotify:artist:2wIVse2owClT7go1WT98tk,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,madeby
3,spotify:artist:2wIVse2owClT7go1WT98tk-spotify:...,spotify:artist:2wIVse2owClT7go1WT98tk,spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,madeby
4,spotify:artist:2wIVse2owClT7go1WT98tk-0,spotify:artist:2wIVse2owClT7go1WT98tk,playlist_0,artistin
...,...,...,...,...
405013,spotify:track:5RVuBrXVLptAEbGJdSDzL5-spotify:a...,spotify:track:5RVuBrXVLptAEbGJdSDzL5,spotify:album:7D6Y19tjm4DQNch39FeWpO,trackin
405014,spotify:artist:6Xa4nbrSTfbioA4lLShbjh-spotify:...,spotify:artist:6Xa4nbrSTfbioA4lLShbjh,spotify:track:5RVuBrXVLptAEbGJdSDzL5,madeby
405015,spotify:artist:6Xa4nbrSTfbioA4lLShbjh-spotify:...,spotify:artist:6Xa4nbrSTfbioA4lLShbjh,spotify:album:7D6Y19tjm4DQNch39FeWpO,madeby
405016,spotify:artist:6Xa4nbrSTfbioA4lLShbjh-999,spotify:artist:6Xa4nbrSTfbioA4lLShbjh,playlist_999,artistin


In [5]:
b_string = df.to_csv(index=False).encode('utf-8')
s3.put_object(Body=b_string, Bucket='bearly-graph', Key='edge/version=2/edges_0.csv')

{'ResponseMetadata': {'RequestId': '63PYAJ8A3SW79Z4R',
  'HostId': '0xnxNX8ZJviymMI3gv28TkI3+JFCb96Gr3MqVuGDPKk9ro0dDJNRRepu4vQyLUzKf9e9p86KwnQ=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '0xnxNX8ZJviymMI3gv28TkI3+JFCb96Gr3MqVuGDPKk9ro0dDJNRRepu4vQyLUzKf9e9p86KwnQ=',
   'x-amz-request-id': '63PYAJ8A3SW79Z4R',
   'date': 'Sun, 07 Nov 2021 04:00:57 GMT',
   'etag': '"e2c8526de8d1f41d4d81b41f86206627"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"e2c8526de8d1f41d4d81b41f86206627"'}